In [1]:
import numpy as np
import pandas as pd
import pyomo.environ as pyo
from measure_optimize import MeasurementOptimizer
import matplotlib.pyplot as plt
from itertools import permutations, product
from scipy.sparse import coo_matrix
from pyomo.contrib.pynumero.interfaces.external_grey_box import ExternalGreyBoxModel
from pyomo.contrib.pynumero.interfaces.external_grey_box import ExternalGreyBoxBlock

## Input Setting

In [2]:
jaco_info = pd.read_csv('./RotaryBed/Q5_scale.csv', index_col=False)

jaco_list = np.asarray(jaco_info)

jaco = []
for i in range(len(jaco_list)):
    jaco.append(list(jaco_list[i][1:]))
    
print(np.shape(jaco))

(70, 5)


In [3]:
number_measure = 14
number_t = 5
number_total = number_measure*number_t
cost = [1]*number_total

## FIM collection and Sigma INV

In [4]:
solution1 = [1]*number_total

In [5]:
calculator = MeasurementOptimizer(jaco, number_measure, number_t, cost, verbose=True)

fim_expect = calculator.fim_computation()

calculator.compute_FIM(solution1)

Q shape: 70 5
======FIM result======
FIM: [[ 2.64284272e+02  9.33719291e-01 -1.84770830e+02  5.17832894e+02
  -4.57843457e+01]
 [ 9.33719291e-01  9.09501280e-02 -3.60408791e-01  3.02000267e+00
  -3.73545512e-01]
 [-1.84770830e+02 -3.60408791e-01  1.46362642e+02 -3.85803017e+02
   1.62152363e+01]
 [ 5.17832894e+02  3.02000267e+00 -3.85803017e+02  1.13614275e+03
  -6.33041909e+01]
 [-4.57843457e+01 -3.73545512e-01  1.62152363e+01 -6.33041909e+01
   2.43541150e+01]]
Determinant: 36029.46902795189 ; log_e(det): 10.492092466690776 ; log_10(det): 4.556657861902482
Trace: 1571.2347270920845 ; log_e(trace): 7.359617039629425 ; log_10(trace): 3.1962410692322054
Min eig: 0.05351208894362994 ; log_e(min_eig): -2.9278476890424687 ; log_10(min_eig): -1.271548095204332
Cond: 28316.222088665123


array([[ 2.64284272e+02,  9.33719291e-01, -1.84770830e+02,
         5.17832894e+02, -4.57843457e+01],
       [ 9.33719291e-01,  9.09501280e-02, -3.60408791e-01,
         3.02000267e+00, -3.73545512e-01],
       [-1.84770830e+02, -3.60408791e-01,  1.46362642e+02,
        -3.85803017e+02,  1.62152363e+01],
       [ 5.17832894e+02,  3.02000267e+00, -3.85803017e+02,
         1.13614275e+03, -6.33041909e+01],
       [-4.57843457e+01, -3.73545512e-01,  1.62152363e+01,
        -6.33041909e+01,  2.43541150e+01]])

In [6]:
from greybox_generalize import LogDetModel

In [7]:
def sgn(p):
    """
    Give the signature of a permutation

    Parameters:
    -----------
    p: the permutation (a list)

    Return:
    ------
    1 if the number of exchange is an even number
    -1 if the number is an odd number
    """

    if len(p) == 1:
        return 1

    trans = 0

    for i in range(0, len(p)):
        j = i + 1

        for j in range(j, len(p)):
            if p[i] > p[j]:
                trans = trans + 1

    if (trans % 2) == 0:
        return 1
    else:
        return -1

In [8]:
#print(np.shape(ans_y))

init_y = np.random.rand(number_total)

update_y = [] 

for i in init_y:
    if i > 0.5: 
        update_y.append(0.99)
    else:
        update_y.append(0.01)

init_cov_y = [[0]*number_total for i in range(number_total)]

for j in range(number_total):
    for k in range(number_total):
        if j==k:
            init_cov_y[j][k] = update_y[k]
            
        else:
            init_cov_y[j][k] = min(update_y[k], update_y[j])

fim_prior = np.identity((5))
print(fim_prior)

[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]


In [9]:
def build_model_external(m):
    ex_model = LogDetModel(num_para=5)
    m.egb = ExternalGreyBoxBlock()
    m.egb.set_external_model(ex_model)

In [10]:
def compute_FIM(numRes, FIMSet, mixed_integer=False,  obj="A", fix=False, budget=100):
    
    m = pyo.ConcreteModel()
    
    m.NumRes = pyo.Set(initialize=range(numRes))
    m.DimFIM = pyo.Set(initialize=range(len(FIMSet[0])))
    
    # set up y and cov y 
    def identity(m, a, b):
        return 1 if a==b else 0
    
    def initialize(m, a, b):
        return init_cov_y[a][b]

    if mixed_integer:
        m.cov_y = pyo.Var(m.NumRes, m.NumRes, within=pyo.Binary)
    else:
        m.cov_y = pyo.Var(m.NumRes, m.NumRes, initialize=initialize, bounds=(0,1), within=pyo.NonNegativeReals)
    
    if fix:
        m.cov_y.fix()
        
    def init_fim(m, p, q):
        return fim_prior[p,q]
    
    # set up FIM
    m.FIM = pyo.Var(m.DimFIM, m.DimFIM, initialize=init_fim)

    # compute FIM
    def eval_fim(m, a, b):
        if a >= b: 
            return m.FIM[a,b] == sum(m.cov_y[i,j]*FIMSet[i*numRes+j][a][b] for i in range(numRes) for j in range(numRes))
        else:
            #return m.TotalFIM[a,b] == sum(m.cov_y[i,j]*FIMSet[i*numRes+j][a][b] for i in range(numRes) for j in range(numRes))
            return m.FIM[a,b] == m.FIM[b,a]
            
    # y and conv y constraints
    def y_covy1(m, a, b):
        if a > b:
            return m.cov_y[a, b] <= m.cov_y[a, a]
        else:
            return pyo.Constraint.Skip
        
    def y_covy2(m, a, b):
        if a > b:
            return m.cov_y[a, b] <= m.cov_y[b, b]
        else:
            return pyo.Constraint.Skip
        
    def y_covy3(m, a, b):
        if a>b:
            return m.cov_y[a, b] >= m.cov_y[a, a] + m.cov_y[b, b] - 1
        else:
            return pyo.Constraint.Skip
    
    def cost_compute(m):
        return m.cost == sum(m.cov_y[i,i] for i in range(numRes))
    
    def cost_limit(m):
        return m.cost <= budget
    
    # set up Design criterion
    def ComputeTrace(m):
        sum_x = sum(m.FIM[j,j] for j in m.DimFIM)
        return sum_x
    
    # add constraints
    m.TotalFIM_con = pyo.Constraint(m.DimFIM, m.DimFIM, rule=eval_fim)
    
    if not fix:
        m.cov1 = pyo.Constraint(m.NumRes, m.NumRes, rule=y_covy1)
        m.cov2 = pyo.Constraint(m.NumRes, m.NumRes, rule=y_covy2)
        m.cov3 = pyo.Constraint(m.NumRes, m.NumRes, rule=y_covy3)
        
        m.cost = pyo.Var(initialize=budget)
        m.cost_compute = pyo.Constraint(rule=cost_compute)
        m.budget_limit = pyo.Constraint(rule=cost_limit)
     
    
    def _model_i(b):
        build_model_external(b)
    m.my_block = pyo.Block(rule=_model_i)
    
    def eq_fim00(m):
        return m.FIM[0,0] == m.my_block.egb.inputs['ele_0_0']
    
    def eq_fim11(m):
        return m.FIM[1,1] == m.my_block.egb.inputs['ele_1_1']
                  
    def eq_fim22(m):
        return m.FIM[2,2] == m.my_block.egb.inputs['ele_2_2']
    
    def eq_fim33(m):
        return m.FIM[3,3] == m.my_block.egb.inputs['ele_3_3']
    
    def eq_fim44(m):
        return m.FIM[4,4] == m.my_block.egb.inputs['ele_4_4']
                  
    m.eq1 = pyo.Constraint(rule=eq_fim00)
    m.eq2 = pyo.Constraint(rule=eq_fim11)
    m.eq3 = pyo.Constraint(rule=eq_fim22)
    m.eq4 = pyo.Constraint(rule=eq_fim33)
    m.eq5 = pyo.Constraint(rule=eq_fim44)
    
    def eq_fim01(m):
        return m.FIM[0,1] == m.my_block.egb.inputs['ele_0_1']
    
    def eq_fim02(m):
        return m.FIM[0,2] == m.my_block.egb.inputs['ele_0_2']
                  
    def eq_fim03(m):
        return m.FIM[0,3] == m.my_block.egb.inputs['ele_0_3']
    
    def eq_fim04(m):
        return m.FIM[0,4] == m.my_block.egb.inputs['ele_0_4']
    
    def eq_fim12(m):
        return m.FIM[1,2] == m.my_block.egb.inputs['ele_1_2']
                  
    m.eq6 = pyo.Constraint(rule=eq_fim01)
    m.eq7 = pyo.Constraint(rule=eq_fim02)
    m.eq8 = pyo.Constraint(rule=eq_fim03)
    m.eq9 = pyo.Constraint(rule=eq_fim04)
    m.eq10 = pyo.Constraint(rule=eq_fim12)
    
    def eq_fim13(m):
        return m.FIM[1,3] == m.my_block.egb.inputs['ele_1_3']
    
    def eq_fim14(m):
        return m.FIM[1,4] == m.my_block.egb.inputs['ele_1_4']
                  
    def eq_fim23(m):
        return m.FIM[2,3] == m.my_block.egb.inputs['ele_2_3']
    
    def eq_fim24(m):
        return m.FIM[2,4] == m.my_block.egb.inputs['ele_2_4']
    
    def eq_fim34(m):
        return m.FIM[3,4] == m.my_block.egb.inputs['ele_3_4']
                  
    m.eq11 = pyo.Constraint(rule=eq_fim13)
    m.eq12 = pyo.Constraint(rule=eq_fim14)
    m.eq13 = pyo.Constraint(rule=eq_fim23)
    m.eq14 = pyo.Constraint(rule=eq_fim24)
    m.eq15 = pyo.Constraint(rule=eq_fim34)


    #fim_values = [m.TotalFIM[0,0], m.TotalFIM[1,1], m.TotalFIM[2,2], m.TotalFIM[3,3], m.TotalFIM[4,4], 
    #     m.TotalFIM[0,1], m.TotalFIM[0,2], m.TotalFIM[0,3], m.TotalFIM[0,4], m.TotalFIM[1,2], 
    #     m.TotalFIM[1,3], m.TotalFIM[1,4], m.TotalFIM[2,3], m.TotalFIM[2,4], m.TotalFIM[3,4]]

    # create a block to store the external grey box model
    #m.my_block = ExternalGreyBoxBlock(external_model=LogDetModel(fim_values))

    # add objective to maximize log det
    m.obj = pyo.Objective(expr=m.my_block.egb.outputs['log_det'], sense=pyo.maximize)

    
    return m 

In [11]:
mod = compute_FIM(number_total, calculator.fim_collection, obj="D", fix=False, budget=100)

solver = pyo.SolverFactory('cyipopt')

solver.config.options['hessian_approximation'] = 'limited-memory' 

additional_options={'max_iter':3000}
for k,v in additional_options.items():
    solver.config.options[k] = v

results = solver.solve(mod, tee=True)


 Consider M =
 [[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]
   logdet =  0.0 


 Consider M =
 [[ 0.3951993   0.3163915   5.37861163 -8.26755306 -4.84608845]
 [ 0.3163915   0.49289527 -0.06073469  0.66298883 -0.19817249]
 [ 5.37861163 -0.06073469 -0.10520699  5.64023025 -3.81162354]
 [-8.26755306  0.66298883  5.64023025  8.03264215  2.86771017]
 [-4.84608845 -0.19817249 -3.81162354  2.86771017  8.46280008]]
   logdet =  6.382558483393479 


 Consider M =
 [[-3.62497659e-01  2.42416105e-01  6.60214573e+00 -1.00698885e+01
  -5.68030262e+00]
 [ 2.42416105e-01  2.82171271e-01 -4.65678884e-04  7.04163299e-01
  -1.83453864e-01]
 [ 6.60214573e+00 -4.65678884e-04 -7.84640283e-01  7.37101716e+00
  -4.32638042e+00]
 [-1.00698885e+01  7.04163299e-01  7.37101716e+00  1.10597109e+01
   2.89604003e+00]
 [-5.68030262e+00 -1.83453864e-01 -4.32638042e+00  2.89604003e+00
   9.67318994e+00]]
   logdet =  6.201308686767504 


 Consider M =
 [[ -0.80101761   0


 Consider M =
 [[ 2.60611631e+02  9.13938830e-01 -1.82463639e+02  5.10964111e+02
  -4.48602762e+01]
 [ 9.13938830e-01  8.85280459e-02 -3.55245417e-01  2.94279507e+00
  -3.58653715e-01]
 [-1.82463639e+02 -3.55245417e-01  1.44375647e+02 -3.80626406e+02
   1.61209919e+01]
 [ 5.10964111e+02  2.94279507e+00 -3.80626406e+02  1.11838971e+03
  -6.24587146e+01]
 [-4.48602762e+01 -3.58653715e-01  1.61209919e+01 -6.24587146e+01
   2.35570752e+01]]
   logdet =  10.300511850090073 


 Consider M =
 [[ 2.62706388e+02  9.24774371e-01 -1.83784170e+02  5.14884201e+02
  -4.53781690e+01]
 [ 9.24774371e-01  8.97588481e-02 -3.57262654e-01  2.98149944e+00
  -3.67255697e-01]
 [-1.83784170e+02 -3.57262654e-01  1.45507655e+02 -3.83556240e+02
   1.61764410e+01]
 [ 5.14884201e+02  2.98149944e+00 -3.83556240e+02  1.12827584e+03
  -6.29487052e+01]
 [-4.53781690e+01 -3.67255697e-01  1.61764410e+01 -6.29487052e+01
   2.39952865e+01]]
   logdet =  10.402588490852281 


 Consider M =
 [[ 2.63537751e+02  9.29511640e-0

In [12]:
total_mea = 14*22
print(total_mea)

308


In [13]:
### FIM
fim_result = np.zeros((5,5))
for i in range(5):
    for j in range(5):
        fim_result[i,j] = pyo.value(mod.FIM[i,j])
        
print(fim_result)  
print('trace:', np.trace(fim_result))
print('det:', np.linalg.det(fim_result))

print("Loge_Det from Pyomo OF:", pyo.value(mod.obj))
print("Loge_Det computing FIM:", np.log(np.linalg.det(fim_result)))

[[ 2.64284272e+02  9.33719277e-01 -1.84770830e+02  5.17832893e+02
  -4.57843450e+01]
 [ 9.33719277e-01  9.09501249e-02 -3.60408784e-01  3.02000257e+00
  -3.73545500e-01]
 [-1.84770830e+02 -3.60408784e-01  1.46362642e+02 -3.85803016e+02
   1.62152365e+01]
 [ 5.17832893e+02  3.02000257e+00 -3.85803016e+02  1.13614274e+03
  -6.33041910e+01]
 [-4.57843450e+01 -3.73545500e-01  1.62152365e+01 -6.33041910e+01
   2.43541140e+01]]
trace: 1571.234717336891
det: 36029.46042543823
Loge_Det from Pyomo OF: 10.492092228532865
Loge_Det computing FIM: 10.492092227927483


In [14]:
fim_aslist = [pyo.value(mod.TotalFIM[0,0]), pyo.value(mod.TotalFIM[1,1]), pyo.value(mod.TotalFIM[2,2]), 
            pyo.value(mod.TotalFIM[3,3]), pyo.value(mod.TotalFIM[4,4]), 
             pyo.value(mod.TotalFIM[0,1]), pyo.value(mod.TotalFIM[0,2]), pyo.value(mod.TotalFIM[0,3]), 
            pyo.value(mod.TotalFIM[0,4]), pyo.value(mod.TotalFIM[1,2]), 
             pyo.value(mod.TotalFIM[1,3]), pyo.value(mod.TotalFIM[1,4]), pyo.value(mod.TotalFIM[2,3]), 
             pyo.value(mod.TotalFIM[2,4]), pyo.value(mod.TotalFIM[3,4])]
print(fim_aslist)

AttributeError: 'ConcreteModel' object has no attribute 'TotalFIM'

In [ ]:
ans_y = np.zeros((number_total,number_total))


#for i in range(70):
    #print('Measurement', i, ':', pyo.value(mod.cov_y[i,i]))
    
for i in range(number_total):
    for j in range(i, number_total):
        cov = pyo.value(mod.cov_y[i,j])
        ans_y[i,j] = cov 
        ans_y[j,i] = pyo.value(mod.cov_y[j,i])
    
print(pyo.value(mod.cost))

In [ ]:
print(ans_y)

figure = plt.figure()
axes = figure.add_subplot(111)

caxes = axes.matshow(ans_y, interpolation='nearest')
figure.colorbar(caxes)

plt.show()

In [ ]:
for i in range(len(ans_y)):
    for j in range(len(ans_y[0])):
        if ans_y[i][j] < 0.05:
            ans_y[i][j] = int(0)
        elif ans_y[i][j] > 0.95:
            ans_y[i][j] = int(1)
        else: 
            ans_y[i][j] = round(ans_y[i][j], 2)
print(ans_y)

In [ ]:
sol_y = np.asarray([ans_y[i,i] for i in range(total_mea)])

sol_y = np.reshape(sol_y, (14,22))
np.around(sol_y)
print(sol_y)

In [ ]:
import pickle

file = open('D_22_100', 'wb')

pickle.dump(ans_y, file)

file.close()

In [ ]:
import matplotlib

def heatmap(data, row_labels, col_labels, ax=None,
            cbar_kw=None, cbarlabel="", **kwargs):
    """
    Create a heatmap from a numpy array and two lists of labels.

    Parameters
    ----------
    data
        A 2D numpy array of shape (M, N).
    row_labels
        A list or array of length M with the labels for the rows.
    col_labels
        A list or array of length N with the labels for the columns.
    ax
        A `matplotlib.axes.Axes` instance to which the heatmap is plotted.  If
        not provided, use current axes or create a new one.  Optional.
    cbar_kw
        A dictionary with arguments to `matplotlib.Figure.colorbar`.  Optional.
    cbarlabel
        The label for the colorbar.  Optional.
    **kwargs
        All other arguments are forwarded to `imshow`.
    """

    if ax is None:
        ax = plt.gca()

    if cbar_kw is None:
        cbar_kw = {}

    # Plot the heatmap
    im = ax.imshow(data, **kwargs)

    # Create colorbar
    cbar = ax.figure.colorbar(im, ax=ax, **cbar_kw)
    cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")

    # Show all ticks and label them with the respective list entries.
    ax.set_xticks(np.arange(data.shape[1]), labels=col_labels)
    ax.set_yticks(np.arange(data.shape[0]), labels=row_labels)

    # Let the horizontal axes labeling appear on top.
    ax.tick_params(top=True, bottom=False,
                   labeltop=True, labelbottom=False)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=-30, ha="right",
             rotation_mode="anchor")

    # Turn spines off and create white grid.
    ax.spines[:].set_visible(False)

    ax.set_xticks(np.arange(data.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
    ax.grid(which="minor", color="w", linestyle='-', linewidth=3)
    ax.tick_params(which="minor", bottom=False, left=False)

    return im, cbar


def annotate_heatmap(im, data=None, valfmt="{x:.2f}",
                     textcolors=("black", "white"),
                     threshold=None, **textkw):
    """
    A function to annotate a heatmap.

    Parameters
    ----------
    im
        The AxesImage to be labeled.
    data
        Data used to annotate.  If None, the image's data is used.  Optional.
    valfmt
        The format of the annotations inside the heatmap.  This should either
        use the string format method, e.g. "$ {x:.2f}", or be a
        `matplotlib.ticker.Formatter`.  Optional.
    textcolors
        A pair of colors.  The first is used for values below a threshold,
        the second for those above.  Optional.
    threshold
        Value in data units according to which the colors from textcolors are
        applied.  If None (the default) uses the middle of the colormap as
        separation.  Optional.
    **kwargs
        All other arguments are forwarded to each call to `text` used to create
        the text labels.
    """

    if not isinstance(data, (list, np.ndarray)):
        data = im.get_array()

    # Normalize the threshold to the images color range.
    if threshold is not None:
        threshold = im.norm(threshold)
    else:
        threshold = im.norm(data.max())/2.

    # Set default alignment to center, but allow it to be
    # overwritten by textkw.
    kw = dict(horizontalalignment="center",
              verticalalignment="center")
    kw.update(textkw)

    # Get the formatter in case a string is supplied
    if isinstance(valfmt, str):
        valfmt = matplotlib.ticker.StrMethodFormatter(valfmt)
        
        
    # Loop over the data and create a `Text` for each "pixel".
    # Change the text's color depending on the data.
    texts = []
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            kw.update(color=textcolors[int(im.norm(data[i, j]) > threshold)])
            text = im.axes.text(j, i, valfmt(data[i, j], None), **kw)
            texts.append(text)

    return texts

In [ ]:
measure_name = ['Ads.gas_inlet.F', 'Ads.gas_outlet.F', 'Ads.gas_outlet.T', 'Ads.gas_outlet.z("CO2")', 'Des.gas_inlet.F', 'Des.gas_outlet.F', 'Des.gas_outlet.T', 'Des.gas_outlet.z("CO2")', 'Ads.T_g.Value(19,10)', 'Ads.T_g.Value(23,10)', 'Ads.T_g.Value(28,10)', 'Ads.z("CO2",19,10)', 'Ads.z("CO2",23,10)', 'Ads.z("CO2",28,10)']

number_t=22
#time_set = ['500', '1000', '1500', '2000', '2200']
time_set = np.linspace(100, 2200, number_t)
time_set = [str(i) for i in time_set]

fig, ax = plt.subplots(figsize=(16,8))

im, cbar = heatmap(sol_y, measure_name, time_set, ax=ax,
                   cmap="YlGn", cbarlabel="weight")
texts = annotate_heatmap(im, valfmt="{x:.1f}")

fig.tight_layout()
plt.show()

In [ ]:
def plot_data(budget, a_cri, d_cri):

    si=18
    plt.rc('axes', titlesize=si)
    plt.rc('axes', labelsize=si)
    plt.rc('xtick', labelsize=si)
    plt.rc('ytick', labelsize=si)

    plt.scatter(budget, np.log10(a_cri), s=50, color='red', label='A-optimality')
    #plt.scatter(budget, np.log10(d_cri), marker='v', s=50, color='blue', label='D-optimality')
    plt.xlabel('Budget', fontweight="bold")
    plt.ylabel('log10(A-optimality)', fontweight="bold")
    
    plt.show()
    
    
    si=18
    plt.rc('axes', titlesize=si)
    plt.rc('axes', labelsize=si)
    plt.rc('xtick', labelsize=si)
    plt.rc('ytick', labelsize=si)

    #plt.scatter(budget, np.log10(a_cri), s=50, color='red', label='A-optimality')
    plt.scatter(budget, np.log10(d_cri), marker='v', s=50, color='blue', label='D-optimality')
    plt.xlabel('Budget', fontweight="bold")
    plt.ylabel('log10(D-optimality)', fontweight="bold")
    
    plt.show()

In [ ]:
a_set = [6940.5857,7058.98522501554, 7102.245, 7117.188, 7121.126]
d_set = [713340241.42,1113638947.8516884, 1317920070.037323, 1414224144.127, 1435685414.35]
plot_data([100,150,200,250,300], a_set, d_set)